In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, LSTM, Bidirectional,  SpatialDropout1D
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
csv_file_path = ''  # Replace with the path to your CSV file
df = pd.read_csv(csv_file_path)

In [ ]:
df = df.rename(columns={'text':'tweets'})

In [ ]:
X=[]
y=[]
for tweet,category in zip(df['tweets'],df['category']):
      X.append(tweet)
      y.append(category)

In [ ]:
tweets=X

Vectorization

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

In [ ]:
import pickle

# Define the path where you want to save the tokenizer
tokenizer_path = ''

# Open a file in binary write mode
with open(tokenizer_path, 'wb') as f:
    pickle.dump(tokenizer, f)

In [ ]:
X_padded = pad_sequences(sequences,maxlen=max_len,padding='post')

In [ ]:
num_classes = len(set(df['category']))

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

Label Encoding

BiLSTM Model

In [ ]:
model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1, 128, input_length=max_len))
model.add(SpatialDropout1D(0.2))  # Dropout layer to prevent overfitting
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

Train Model

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)

In [ ]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred,average='macro'))
print("Precision:", precision_score(y_test, y_pred,average='macro'))
print("F1:", f1_score(y_test, y_pred,average='macro'))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

cf=confusion_matrix(y_test,y_pred)
plt.figure(figsize=(12, 12))  # Adjust the figure size accordingly
sns.set(font_scale=0.5)  # Adjust font scale for labels
sns.heatmap(cf, annot=True, fmt='d', cmap='Blues', cbar_kws={'label': 'Scale'}, square=True)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix Heatmap BiLSTM')
plt.show()

In [ ]:
model.save('') 